In [64]:
import os
import pandas as pd
import librosa
import glob
import numpy as np
import xgboost as xgb

# 处理训练集

In [70]:
path = '初赛训练集\\cough'
fea = []
label = []
for i in os.listdir(path):
    for j in os.listdir(label_path+'\\'+i):
        y, sr = librosa.load(path=label_path+'\\'+i+'\\'+j, sr=None, mono=False)
        y = y[::3]
        # 默认提取 20 帧
        audio_mac = librosa.feature.mfcc(y=y, sr=16000)
        y_shape = audio_mac.shape[1]
        max_pad_size=11
        if y_shape < max_pad_size:
            pad_size = max_pad_size - y_shape
            audio_mac = np.pad(audio_mac, ((0, 0), (0, pad_size)), mode='constant')
        else:
            audio_mac = audio_mac[:, :max_pad_size]
        fea.append(audio_mac.flatten())
        label.append(i)

In [71]:
df = pd.DataFrame(fea)
df['label'] = label
fea_names = [i for i in df.columns if i not in ['label']]

In [72]:
model = xgb.XGBClassifier()
model.fit(df[fea_names],df['label'])

[23:14:32] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

# 处理测试集

In [55]:
path = '初赛测试集'
fea = []
files = []
for j in os.listdir(path):
    files.append(j)
    y, sr = librosa.load(path=path+'\\'+j, sr=None, mono=False)
    y = y[::3]
    # 默认提取 20 帧
    audio_mac = librosa.feature.mfcc(y=y, sr=16000)
    y_shape = audio_mac.shape[1]
    max_pad_size=11
    if y_shape < max_pad_size:
        pad_size = max_pad_size - y_shape
        audio_mac = np.pad(audio_mac, ((0, 0), (0, pad_size)), mode='constant')
    else:
        audio_mac = audio_mac[:, :max_pad_size]
    fea.append(audio_mac.flatten())

In [56]:
df2 = pd.DataFrame(fea)
df2['category_id'] = model.predict(df2[fea_names])
df2['category_id'].value_counts()

In [78]:
df2['sample_id']=files
df2[['sample_id','category_id']].to_csv('sub.csv',index=False)

In [ ]:
# 使用keras或其他方法
# df['label'] = df['label'].apply(lambda x:0 if x=='Negative' else 1)

# from sklearn.model_selection import train_test_split
# x_train, x_val, y_train, y_val = train_test_split(df[fea_names], df['label'], test_size=0.2, random_state=42)

# from keras.models import Sequential
# from keras.layers import Dense
# import keras

# model = Sequential()
# model.add(Dense(64, activation='relu', input_shape=(220,)))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(1, activation='softmax'))

# model.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.RMSprop(),
#               metrics=['accuracy'])
# model.fit(x_train, y_train, batch_size=30, epochs=20, verbose=1,validation_data=(x_val, y_val))